In [96]:
p_storage_dir = "../../storage"

In [97]:
import unicodedata
import re
from pathlib import Path

import pandas as pd
from thefuzz import fuzz

IFG_PRODUZ_STORAGE_DIR = Path(f"{p_storage_dir}/ifg_produz")

## Unidades Federativas

In [98]:
ufs = pd.read_csv(IFG_PRODUZ_STORAGE_DIR / "intermediate/unidades_federativas.csv", delimiter=";")

ufs.sample(n=5)

,sigla
4,CE
12,PA
19,RO
18,RN
0,AC


**Colocar o nome completo em cada UF**

In [99]:
ufs['nome'] = ufs['sigla']

In [100]:
uf_name_replacement_map = {
    'AC': 'Acre',
    'AL': 'Alagoas',
    'AM': 'Amazonas',
    'BA': 'Bahia',
    'CE': 'Ceará',
    'DF': 'Distrito Federal',
    'ES': 'Espírito Santo',
    'GO': 'Goiás',
    'MA': 'Maranhão',
    'MG': 'Minas Gerais',
    'MS': 'Mato Grosso do Sul',
    'MT': 'Mato Grosso',
    'PA': 'Pará',
    'PB': 'Paraíba',
    'PE': 'Pernambuco',
    'PI': 'Piauí',
    'PR': 'Paraná',
    'RJ': 'Rio de Janeiro',
    'RN': 'Rio Grande do Norte',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'RS': 'Rio Grande do Sul',
    'SC': 'Santa Catarina',
    'SE': 'Sergipe',
    'SP': 'São Paulo',
    'TO': 'Tocantins'
}

In [101]:
ufs['nome'] = ufs['nome'].replace(uf_name_replacement_map)

In [102]:
ufs.sample(n=10)

,sigla,nome
22,SC,Santa Catarina
23,SE,Sergipe
14,PE,Pernambuco
15,PI,Piauí
5,DF,Distrito Federal
7,GO,Goiás
3,BA,Bahia
16,PR,Paraná
12,PA,Pará
2,AM,Amazonas


## Cidades

In [103]:
cidades = pd.read_csv(IFG_PRODUZ_STORAGE_DIR / "intermediate/cidades.csv", delimiter=";")

cidades.head()

,nome,sigla_uf
0,ALMEIRIM,PA
1,ANÁPOLIS,GO
2,APARECIDA DE GOIÂNIA,GO
3,Abadiânia,GO
4,Alagoinhas,BA


**Normalizar o nome das cidades**

In [104]:
cidades['nome_normalizado'] = cidades['nome']

In [105]:
def normalizar_string(s):
    # Transforma todos os caracteres em minúsculas
    s = s.lower()
    
    # Remove acentos das letras
    s = unicodedata.normalize("NFD", s).encode("ascii", "ignore").decode("utf-8")

    # Substitui todos os caracteres que não são letras por um espaço em branco
    s = re.sub(r'[^A-Za-z ]', ' ', s)

    # Substitui espaços consecutivos por apenas um espaço
    s = re.sub(r'\d{2,}', ' ', s)

    # Remove os espaços em branco do começo e fim
    s = s.strip()

    return s

In [106]:
cidades['nome_normalizado'] = cidades['nome_normalizado'].apply(normalizar_string)

In [107]:
cidades[cidades['nome_normalizado'].str.contains(' ')].head(n=10)

,nome,sigla_uf,nome_normalizado
2,APARECIDA DE GOIÂNIA,GO,aparecida de goiania
6,Alto Araguaia,MT,alto araguaia
7,Alto Garças,MT,alto garcas
8,Amarante do Maranhão,MA,amarante do maranhao
14,Aparecida de Goiânia,GO,aparecida de goiania
15,Aparecida de goiânia,GO,aparecida de goiania
29,BELO HORIZONTE,MG,belo horizonte
31,BOA VISTA,RR,boa vista
33,BRASILIA-DF,DF,brasilia df
36,Barra do Corda,MA,barra do corda


### Reconciliar as cidades por UF

In [108]:
cidades.groupby('sigla_uf')['nome'].count().sort_values(ascending=False)

sigla_uf
GO    153
MG     65
SP     55
RS     17
BA     13
DF     12
MA     11
MT     11
RJ     11
PR     10
TO     10
PA      7
MS      6
PE      6
PI      5
ES      4
CE      4
SC      3
PB      3
RN      2
SE      2
RO      1
RR      1
AL      1
AM      1
AC      1
Name: nome, dtype: int64

**Funções auxiliares**

In [109]:
def get_first_token_length(s):
    partes = s.split(" ")
    return len(partes[0])

**Rio Grande do Sul - RS**

In [110]:
cidades_rs = cidades[cidades['sigla_uf'] == 'RS']

In [111]:
cidades_rs.shape

(17, 3)

In [112]:
cidades_rs.columns

Index(['nome', 'sigla_uf', 'nome_normalizado'], dtype='object')

In [113]:
nome_cidades_rs = [
    'Bagé', 'Boa Vista do Buricá', 'Catuípe', 'Ijuí', 'Independência',
    'Muçum', 'Pelotas', 'Porto Alegre', 'Rio Grande', 'Santa Maria',
    'Santiago', 'Santo Ângelo', 'São Francisco de Assis',
    'Tenente Portela', 'Uruguaiana', 'Venâncio Aires'
]


nome_ascii_cidades_rs = [
    'bage', 'boa vista do burica', 'catuipe', 'ijui', 'independencia',
    'mucum', 'pelotas', 'porto alegre', 'rio grande', 'santa maria',
    'santiago', 'santo angelo', 'sao francisco de assis',
    'tenente portela', 'uruguaiana', 'venancio aires'
]

In [114]:
cidades_rs_final = pd.DataFrame({'nome': nome_cidades_rs, 'nome_ascii': nome_ascii_cidades_rs, 'sigla_uf': ['RS'] * len(nome_cidades_rs)})

In [115]:
cidades_rs_final.sample(n=5)

,nome,nome_ascii,sigla_uf
11,Santo Ângelo,santo angelo,RS
7,Porto Alegre,porto alegre,RS
0,Bagé,bage,RS
8,Rio Grande,rio grande,RS
10,Santiago,santiago,RS


In [116]:
cidades_rs.loc[:, ['tamanho_primeiro_token']] = cidades_rs['nome_normalizado'].apply(get_first_token_length)

In [117]:
cidades_rs['tamanho_primeiro_token'].min()

3

In [118]:
nome_ascii_to_nome = list(zip(cidades_rs["nome"], cidades_rs["nome_normalizado"]))

In [119]:
def create_aliases(row):
    nome_ascii = row["nome_ascii"]
    aliases = [nome_desnormalizado for nome_desnormalizado, nome_normalizado in
               nome_ascii_to_nome if fuzz.ratio(nome_normalizado, nome_ascii) >= 80 and nome_normalizado[:3] == nome_ascii[:3]]
    return aliases

In [120]:
cidades_rs_final['aliases'] = cidades_rs_final.apply(create_aliases, axis=1)

In [121]:
cidades_rs_final

,nome,nome_ascii,sigla_uf,aliases
0,Bagé,bage,RS,[BAGÉ]
1,Boa Vista do Buricá,boa vista do burica,RS,[Boa Vista do Buricá]
2,Catuípe,catuipe,RS,[Catuípe]
3,Ijuí,ijui,RS,[Ijuí]
4,Independência,independencia,RS,[Independência]
5,Muçum,mucum,RS,[MUÇUM]
6,Pelotas,pelotas,RS,[Pelotas]
7,Porto Alegre,porto alegre,RS,[Porto Alegre]
8,Rio Grande,rio grande,RS,[Rio Grande]
9,Santa Maria,santa maria,RS,"[Santa Maria, santa maria]"


**Bahia - BA**

In [122]:
cidades_ba = cidades[cidades['sigla_uf'] == 'BA']

In [123]:
cidades_ba

,nome,sigla_uf,nome_normalizado
4,Alagoinhas,BA,alagoinhas
46,Bom Jesus da Serra,BA,bom jesus da serra
48,Brasilia,BA,brasilia
84,Casa Nova,BA,casa nova
95,Cocos,BA,cocos
98,Correntina,BA,correntina
181,Itaberaba,BA,itaberaba
184,Itabuna,BA,itabuna
186,Itamaraju,BA,itamaraju
202,Jacobina,BA,jacobina


In [124]:
cidades_ba = cidades_ba.drop(cidades_ba[cidades_ba['nome_normalizado'] == 'brasilia'].index)

In [125]:
len(cidades_ba['nome_normalizado'].unique()) == len(cidades_ba['nome_normalizado'])

True

In [126]:
cidades_ba_final = cidades_ba[['nome', 'sigla_uf']]

In [127]:
cidades_ba_final['aliases'] = cidades_ba['nome'].apply(lambda s: [s])

In [128]:
cidades_ba_final['nome_ascii'] = cidades_ba['nome_normalizado']

In [129]:
cidades_ba_final

,nome,sigla_uf,aliases,nome_ascii
4,Alagoinhas,BA,[Alagoinhas],alagoinhas
46,Bom Jesus da Serra,BA,[Bom Jesus da Serra],bom jesus da serra
84,Casa Nova,BA,[Casa Nova],casa nova
95,Cocos,BA,[Cocos],cocos
98,Correntina,BA,[Correntina],correntina
181,Itaberaba,BA,[Itaberaba],itaberaba
184,Itabuna,BA,[Itabuna],itabuna
186,Itamaraju,BA,[Itamaraju],itamaraju
202,Jacobina,BA,[Jacobina],jacobina
211,Jequié,BA,[Jequié],jequie


**Distrito Federal - DF**

In [130]:
cidades_df = cidades[cidades['sigla_uf'] == 'DF']

In [131]:
cidades_df

,nome,sigla_uf,nome_normalizado
32,BRASILIA,DF,brasilia
33,BRASILIA-DF,DF,brasilia df
49,Braslândia,DF,braslandia
50,Brasília,DF,brasilia
56,CEILÂNDIA,DF,ceilandia
135,Gama,DF,gama
310,Recanto das Emas,DF,recanto das emas
347,Sobradinho,DF,sobradinho
348,Sobradinho - DF,DF,sobradinho df
373,Taguatinga,DF,taguatinga


In [132]:
nome_cidades_df = [
    'Brasília', 'Brazlândia', 'Ceilândia', 'Gama', 'Recanto das Emas', 'Sobradinho', 'Taguatinga'
]


nome_ascii_cidades_df = [
    'brasilia', 'brazlandia', 'ceilandia', 'gama', 'recanto das emas', 'sobradinho', 'taguatinga'
]

In [133]:
cidades_df_final = pd.DataFrame({'nome': nome_cidades_df, 'nome_ascii': nome_ascii_cidades_df, 'sigla_uf': ['DF'] * len(nome_cidades_df)})

In [134]:
cidades_df['nome_normalizado'].apply(get_first_token_length).min()

4

In [135]:
nome_ascii_to_nome = list(zip(cidades_df['nome'], cidades_df['nome_normalizado']))

In [136]:
def create_aliases(row):
    nome_ascii = row["nome_ascii"]
    aliases = [nome_desnormalizado for nome_desnormalizado, nome_normalizado in
               nome_ascii_to_nome if fuzz.ratio(nome_normalizado, nome_ascii) >= 80 and nome_normalizado[:2] == nome_ascii[:2]]
    return aliases

In [137]:
cidades_df_final['aliases'] = cidades_df_final.apply(create_aliases, axis=1)

In [138]:
cidades_df_final

,nome,nome_ascii,sigla_uf,aliases
0,Brasília,brasilia,DF,"[BRASILIA, BRASILIA-DF, Brasília, brasilia]"
1,Brazlândia,brazlandia,DF,[Braslândia]
2,Ceilândia,ceilandia,DF,[CEILÂNDIA]
3,Gama,gama,DF,[Gama]
4,Recanto das Emas,recanto das emas,DF,[Recanto das Emas]
5,Sobradinho,sobradinho,DF,"[Sobradinho, Sobradinho - DF]"
6,Taguatinga,taguatinga,DF,"[Taguatinga, Taguatinga Sul]"


**Maranhão - MA**

In [139]:
cidades_ma = cidades[cidades['sigla_uf'] == 'MA']

In [140]:
cidades_ma

,nome,sigla_uf,nome_normalizado
8,Amarante do Maranhão,MA,amarante do maranhao
36,Barra do Corda,MA,barra do corda
82,Carolina,MA,carolina
150,Grajaú,MA,grajau
164,Icatu,MA,icatu
170,Imperatriz,MA,imperatriz
189,Itapecuru-Mirim,MA,itapecuru mirim
332,Santa Luzia,MA,santa luzia
363,São Luis,MA,sao luis
366,São Luís,MA,sao luis


In [141]:
nome_cidades_ma = [
    'Amarante do Maranhão', 'Barra do Corda', 'Carolina', 'Grajaú', 'Icatu', 'Imperatriz', 'Itapecuru-Mirim',
    'Santa Luzia', 'São Luís', 'Viana'
]


nome_ascii_cidades_ma = [
    'amarante do maranhao', 'barra do corda', 'carolina', 'grajau', 'icatu', 'imperatriz', 'itapecuru-mirim',
    'santa luzia', 'sao luis', 'viana'
]

In [142]:
cidades_ma_final = pd.DataFrame({'nome': nome_cidades_ma, 'nome_ascii': nome_ascii_cidades_ma, 'sigla_uf': ['MA'] * len(nome_cidades_ma)})

In [143]:
cidades_ma_final

,nome,nome_ascii,sigla_uf
0,Amarante do Maranhão,amarante do maranhao,MA
1,Barra do Corda,barra do corda,MA
2,Carolina,carolina,MA
3,Grajaú,grajau,MA
4,Icatu,icatu,MA
5,Imperatriz,imperatriz,MA
6,Itapecuru-Mirim,itapecuru-mirim,MA
7,Santa Luzia,santa luzia,MA
8,São Luís,sao luis,MA
9,Viana,viana,MA


In [144]:
nome_ascii_to_nome = list(zip(cidades_ma['nome'], cidades_ma['nome_normalizado']))

In [145]:
def create_aliases(row):
    nome_ascii = row["nome_ascii"]
    aliases = [nome_desnormalizado for nome_desnormalizado, nome_normalizado in
               nome_ascii_to_nome if fuzz.ratio(nome_normalizado, nome_ascii) >= 85]
    return aliases

In [146]:
cidades_ma_final['aliases'] = cidades_ma_final.apply(create_aliases, axis=1)

In [147]:
cidades_ma_final

,nome,nome_ascii,sigla_uf,aliases
0,Amarante do Maranhão,amarante do maranhao,MA,[Amarante do Maranhão]
1,Barra do Corda,barra do corda,MA,[Barra do Corda]
2,Carolina,carolina,MA,[Carolina]
3,Grajaú,grajau,MA,[Grajaú]
4,Icatu,icatu,MA,[Icatu]
5,Imperatriz,imperatriz,MA,[Imperatriz]
6,Itapecuru-Mirim,itapecuru-mirim,MA,[Itapecuru-Mirim]
7,Santa Luzia,santa luzia,MA,[Santa Luzia]
8,São Luís,sao luis,MA,"[São Luis, São Luís]"
9,Viana,viana,MA,[Viana]
